<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

[DC fork](https://github.com/dcsan/dalle-playground)

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [ ]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!npm install -g localtunnel

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [ ]:
# check run time type to see if we have a GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mega_full' #@param ["Mini", "Mega", "Mega_full"]

# Run the backend web server

In [ ]:
from threading import Thread

def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py 8000 $dalle_model

if __name__ == '__main__':
    t1 = Thread(target = app)
    a = t1.start()
    !lt --port 8000

Selected DALL-E Model - [Mega]
--> Starting DALL-E Server. This might take up to two minutes.
your url is: https://metal-points-itch-35-231-76-21.loca.lt
tcmalloc: large alloc 5175050240 bytes == 0xbcc6000 @  0x7f61f57841e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x4bc98a 0x532b86 0x594a96 0x515600 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f61f5381c87 0x5b621a
Some of the weights of DalleBart were initialized in float16 precision from the model checkpoint at /tmp/tmpszd3x2ts:
[('lm_head', 'kernel'), ('model', 'decoder', 'embed_positions', 'embedding'), ('model', 'decoder', 'embed_tokens', 'embedding'), ('model', 'decoder', 'final_ln', 'bias'), ('model', 'decoder', 'layernorm_embedding', 'bias'), ('model', 'decoder', 'layernorm_embedding', 'scale'), ('model', 'decoder', 'layers', 'FlaxBartDecoderLayers', 'FlaxBartAttention_0', 'k_proj', 'kernel'), ('mo

### Now, take the url you got from localtunnel (should look like `your url is: https://xxxxxx.loca.lt`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.loca.lt and run it in the browser. 

### Let the fun begin ✨